# Intento con .txt

Se uso Acrobat para exportar el pdf a formato .txt

https://repositorio.minedu.gob.pe/browse?value=Quechua+Chanka&type=subject

In [8]:
#import os
#os.chdir("./data/txt")
#print(os.getcwd())
!ls

Amy.ipynb
CuadernoPrincipal.ipynb
Harvy.ipynb
README.md
Renzo.ipynb
data


In [9]:
!ls

Amy.ipynb
CuadernoPrincipal.ipynb
Harvy.ipynb
README.md
Renzo.ipynb
data


In [ ]:
#key="2 Rimana - Qillqasqa maytu - Qichwa, Chanka. Texto de Comunicación del 2° Secundaria - Quechua chanka"
key="Anqarakunapa kawsakuyninmanta. Literatura 2 - 4° Primaria - Quechua chanka"

path = f'./data/txt/{key}.txt'

try:
    with open(path, 'r', encoding='ISO-8859-1') as file:
        text = file.read()
except UnicodeDecodeError:
    with open(path, 'r', encoding='Windows-1252') as file:
        text = file.read()
text

In [ ]:
import re

# Eliminar saltos de línea y tabulaciones
text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ')

# Opcional: Eliminar caracteres no deseados
text = re.sub(r'[^a-zA-Z0-9.,;:!?() ]', '', text)

# Eliminar espacios extra
text = re.sub(r'\s+', ' ', text).strip()

# Pasar a minúsculas
text = text.lower()

In [ ]:
type(text)

str

Análisis básico

In [ ]:
# Contar palabras
words = text.split()
num_words = len(words)
print("Número de palabras:", num_words)

# Contar frases (asumiendo que cada punto final representa el fin de una frase)
sentences = text.split('.')
num_sentences = len(sentences)
print("Número de frases:", num_sentences)

# Frecuencia de palabras
from collections import Counter
frequency = Counter(words)
print("Las diez palabras más comunes:", frequency.most_common(10))


Número de palabras: 14938
Número de frases: 1008
Las diez palabras más comunes: [('de', 586), ('los', 392), ('y', 316), ('la', 294), ('en', 253), ('el', 252), ('que', 183), ('con', 159), ('a', 155), ('se', 143)]


# Filtrado de oraciones en español

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from langdetect import detect
nltk.download('punkt')

sentences = sent_tokenize(text)
non_spanish_sentences = []

#Oraciones antes de filtrar
print("Oraciones antes de filtrar: ", len(sentences))
for sentence in sentences:
    try:
        if detect(sentence) != 'es':
            non_spanish_sentences.append(sentence)
    except:
        pass

sentences = non_spanish_sentences
print("Oraciones después de filtrar: ", len(sentences))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\renzo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Oraciones antes de filtrar:  996
Oraciones después de filtrar:  473


In [7]:
# Guardar las oraciones no españolas en un archivo
with open(f'./data/corpus/{key}.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\n')


In [6]:
#a, aa, ch, chh, ch', ts, tr, h, i, ii, k, kh, k', l, ll, m, n, ñ, p, ph, p', q, qh, q', r, s, sh, t, th, t', u, uu, w, y
alfabeto_quechua = ['a', 'aa', 'ch', 'chh', 'ch\'', 'ts', 'tr', 'h', 'i', 'ii', 'k', 'kh', 'k\'', 'l', 'll', 'm', 'n', 'ñ', 'p', 'ph', 'p\'', 'q', 'qh', 'q\'', 'r', 's', 'sh', 't', 'th', 't\'', 'u', 'uu', 'w', 'y']

def rule_based_heuristic(sentence):
    #exclude sentences containing words formed by graphemes outside the alphabet
    words = word_tokenize(sentence)
    for word in words:
        if word not in alfabeto_quechua:
            return False
        
    return True

# Con PDF (copia Harvy)


In [ ]:
from pdfminer.high_level import extract_text
from pypdf import PdfReader, PdfWriter
import fitz
import pandas as pd

documents={
    "2 Rimana - Qillqasqa Mayt’u Qichwa Qullaw. Texto de Comunicación del 2° Secundaria - Quechua Collao": (4, 6),
    #"Anqarakunapa kawsakuyninmanta. Literatura 2 - 4° Primaria - Quechua chanka": (5,6),
    #"Antuku hampatuwan. Historias y relatos 2- Inicial - Quechua chanka": (4, 3),
    "Ayllupi yachasunchik 3 ñiqi Qullaw qichwapi. Texto de Comunicación del 3° Primaria - Quechua Collao": (4,3),
    "Huk kutis kaq kasqa Literatura 2 quechua collao": (4,5),
    "Huk kutis kaq kasqa. Literatura 1 - 5° Primaria - Quechua collao": (4,5),
    "Llaqtanchikpa kawsayninkuna Saberes de los pueblos 2 - 2° Secundaria - Quechua collao": (4, 5),
    "Llaqtanchikpa kawsayninkuna. Saberes de los pueblos 1 - 1° Secundaria - Quechua collao": (4, 5)
}

In [ ]:
import re


def get_text_from_pdf(key):
    doc = fitz.open(f'data/{key}.pdf')
    number_of_pages = doc.page_count

    i=documents[key][0]
    full_text=""
    for page in doc.pages(documents[key][0], number_of_pages-documents[key][1]):
        # print("Page number: ", i)
        this_page_text=page.get_text("text", sort=True).replace("\n", " ")
        this_page_text=re.sub(r'\d+', '', this_page_text)
        full_text+=this_page_text+" "
        i+=1
    return full_text

In [ ]:
from nltk.tokenize import sent_tokenize
from langdetect import detect
import langid

def filtrar_espaniol(full_text):
    sentences = sent_tokenize(full_text)
    non_spanish_sentences = []

    
    for sentence in sentences:
        try:
            languages=[]
            languages.append(detect(sentence))
            languages.append(langid.classify(sentence)[0])
            if not any(lang in languages for lang in ["es", "it", "ca", "pt"]):
                non_spanish_sentences.append(sentence)
        except:
            pass

    sentences = non_spanish_sentences
    

    return sentences

def filtrar_oracion_espaniol(sentence):
    try:
        languages=[]
        languages.append(detect(sentence))
        languages.append(langid.classify(sentence)[0])
        if not any(lang in languages for lang in ["es", "it", "ca", "pt"]):
            return True
    except:
        pass

    return False

In [ ]:
#Funciones de filtrado
alfabeto_quechua = ['a', 'aa', 'ch', 'chh', 'ch\'', 'ts', 'tr', 'h', 'i', 'ii', 'k', 'kh', 'k\'', 'l', 'll', 'm', 'n', 'ñ', 'p', 'ph', 'p\'', 'q', 'qh', 'q\'', 'r', 's', 'sh', 't', 'th', 't\'', 'u', 'uu', 'w', 'y']

def grafemas_no_en_alfabet(words):
    for word in words:
        for i, letter in enumerate(word):
            #Continue  if letter is not a letter
            if not letter.isalpha():
                continue
            if letter.lower() not in alfabeto_quechua:
                #Chequear siguiente letra
                if i+1 >= len(word):
                    return False
                letter = letter + word[i+1]
                if letter.lower() not in alfabeto_quechua:
                    if i+2 >= len(word):
                        return False
                    #Chequear siguiente letra
                    letter = letter + word[i+2]
                    if letter.lower() not in alfabeto_quechua:
                        return False
    return True

def oraciones_mucho_espaniol(words):
    spanish_words = 0
    for word in words:
        try:
            if detect(word) == 'es':
                spanish_words += 1
        except:
            pass
    return spanish_words/len(words) > 0.5

def oraciones_muy_cortas(words, min_length=3):
    #Conservar solo palabras que no sean puntuación
    aux = [word for word in words if word.isalpha()]
    return len(aux) > min_length

def oraciones_muy_repititivas(words, threshold=0.4):
    unique_words = set(words)
    ratio = len(unique_words) / len(words)
    return ratio >= threshold

def palabras_muy_largas(words, threshold=40):
    for word in words:
        if len(word) > threshold:
            return False
    return True

def split_tokens(sentence):
    # Check for three or more sequential tokens composed of one or two characters
    if re.search(r"(\b\w{1,2}\b\s){3,}", sentence):
        return False
    return True

def oraciones_con_matematica(sentence):
    if re.search(r"[\d+\-*/]+", sentence):
        return False
    return True

In [ ]:
#a, aa, ch, chh, ch', ts, tr, h, i, ii, k, kh, k', l, ll, m, n, ñ, p, ph, p', q, qh, q', r, s, sh, t, th, t', u, uu, w, y
from nltk.tokenize import word_tokenize
# Reglas basadas en https://aclanthology.org/2020.lrec-1.356/
def rule_based_heuristic(sentence):
    words = word_tokenize(sentence)
    
    valid = oraciones_muy_cortas(words) 
    valid = valid and oraciones_muy_repititivas(words)
    valid = valid and palabras_muy_largas(words)
    valid = valid and split_tokens(sentence)
    valid = valid and oraciones_con_matematica(sentence)

    return valid

In [ ]:
def pipeline(documents):
    corpus=pd.DataFrame(columns=["document", "sentence"])
    for key, value in documents.items():
        text=get_text_from_pdf(key)
        sentences=sent_tokenize(text)
        #Oraciones antes de filtrar
        print("Oraciones antes de filtrar en documento ", key, ": ", len(sentences))
        filtered_sentences=[]
        for sentence in sentences:
            if rule_based_heuristic(sentence) and filtrar_oracion_espaniol(sentence):
                filtered_sentences.append(sentence)
        
        df=pd.DataFrame(filtered_sentences, columns=["sentence"])
        df["document"]=key
        #Quitar duplicados
        df.drop_duplicates(subset="sentence", inplace=True)
        print("Oraciones después de filtrar en documento ", key, ": ", len(df))
        corpus=pd.concat([corpus, df], ignore_index=True)
    print("Total de oraciones: ", len(corpus))
    corpus.drop_duplicates(subset="sentence", inplace=True)
    print("Total de oraciones únicas: ", len(corpus))
    return corpus

In [ ]:
df_corpus=pipeline(documents)
df_corpus.head()

In [ ]:
#Show me the sortest sentences  content
df_corpus.loc[df_corpus["sentence"].apply(lambda x: len(x))<40]

In [ ]:
df_corpus.to_csv("data/corpus/avances/avanceRenzo.csv", index=False)